In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 200     # number of variables
num_ineq = 200    # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msNonconvex
model = msNonconvex(num_var, num_ineq, timelimit=60)

## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 512           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmNonconvex
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmNonconvex(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 9256.14
Epoch 1, Validation Loss: 22.58
Epoch 2, Validation Loss: 13.68
Epoch 3, Validation Loss: 10.55
Epoch 4, Validation Loss: 8.57
Epoch 5, Validation Loss: 6.48
Epoch 6, Validation Loss: 5.98
Epoch 7, Validation Loss: 5.64
Epoch 8, Validation Loss: 5.94
Epoch 9, Validation Loss: 4.52
Epoch 10, Validation Loss: 3.94
Epoch 11, Validation Loss: 3.48
Epoch 12, Validation Loss: 3.19
Epoch 13, Validation Loss: 3.40
Epoch 14, Validation Loss: 3.06
Epoch 15, Validation Loss: 2.91
Epoch 16, Validation Loss: 2.35
Epoch 17, Validation Loss: 3.85
Epoch 18, Validation Loss: 2.64
Epoch 19, Validation Loss: 2.57
Epoch 20, Validation Loss: 4.44
Epoch 21, Validation Loss: 2.81
Epoch 22, Validation Loss: 2.47
Epoch 23, Validation Loss: 3.29
Epoch 24, Validation Loss: 2.09
Epoch 25, Validation Loss: 2.46
Epoch 26, Validation Loss: 2.32
Epoch 27, Validation Loss: 2.15
Epoch 28, Validation Loss: 1.83
Epoch 29, Validation Loss: 2.08
Epoch 30, Validation Loss: 2.98
Epoch 31, Va

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_lr_200-200.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.18it/s]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+02             100.0    100.000000
mean   1.688158e+00               0.0      0.003859
std    1.785306e-15               0.0      0.000890
min    1.688158e+00               0.0      0.002501
25%    1.688158e+00               0.0      0.003006
50%    1.688158e+00               0.0      0.003654
75%    1.688158e+00               0.0      0.004514
max    1.688158e+00               0.0      0.006001
Number of infeasible solution: 0


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 512           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmNonconvex
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmNonconvex(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 5004.20
Epoch 1, Validation Loss: 9.51
Epoch 2, Validation Loss: 7.14
Epoch 3, Validation Loss: 7.16
Epoch 4, Validation Loss: 4.09
Epoch 5, Validation Loss: 3.89
Epoch 6, Validation Loss: 3.55
Epoch 7, Validation Loss: 3.46
Epoch 8, Validation Loss: 4.30
Epoch 9, Validation Loss: 2.86
Epoch 10, Validation Loss: 2.16
Epoch 11, Validation Loss: 2.26
Epoch 12, Validation Loss: 2.58
Epoch 13, Validation Loss: 2.12
Epoch 14, Validation Loss: 3.22
Epoch 15, Validation Loss: 2.27
Epoch 16, Validation Loss: 2.54
Epoch 17, Validation Loss: 1.00
Epoch 18, Validation Loss: 2.38
Epoch 19, Validation Loss: 2.63
Epoch 20, Validation Loss: 1.99
Epoch 21, Validation Loss: 3.37
Epoch 22, Validation Loss: 1.58
Epoch 23, Validation Loss: 1.82
Epoch 24, Validation Loss: 1.54
Epoch 25, Validation Loss: 1.98
Epoch 26, Validation Loss: 2.08
Epoch 27, Validation Loss: 3.12
Epoch 28, Validation Loss: 1.00
Epoch 29, Validation Loss: 1.26
Epoch 30, Validation Loss: 1.05
Epoch 31, Valid

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_lt_200-200.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:10<00:00,  9.35it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000        100.000000    100.000000
mean     0.514225          0.000066      0.003275
std      0.174481          0.000658      0.000786
min      0.331343          0.000000      0.001998
25%      0.439910          0.000000      0.002790
50%      0.464498          0.000000      0.003013
75%      0.531973          0.000000      0.003548
max      1.430887          0.006576      0.005945
Number of infeasible solution: 1


## Parametric Learning Then Rounding

In [17]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [18]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 512           # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [19]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# build neuromancer problem for rounding
components = nn.ModuleList([smap]).to("cuda")
loss_fn = nmQuadratic(["b", "x"], num_var, num_ineq, penalty_weight)

In [20]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 5051.07
Epoch 1, Validation Loss: -7.04
Epoch 2, Validation Loss: -14.87
Epoch 3, Validation Loss: -21.93
Epoch 4, Validation Loss: -25.11
Epoch 5, Validation Loss: -26.60
Epoch 6, Validation Loss: -29.16
Epoch 7, Validation Loss: -29.11
Epoch 8, Validation Loss: -29.42
Epoch 9, Validation Loss: -30.08
Epoch 10, Validation Loss: -29.61
Epoch 11, Validation Loss: -30.43
Epoch 12, Validation Loss: -31.61
Epoch 13, Validation Loss: -31.18
Epoch 14, Validation Loss: -31.79
Epoch 15, Validation Loss: -32.95
Epoch 16, Validation Loss: -32.41
Epoch 17, Validation Loss: -32.69
Epoch 18, Validation Loss: -32.00
Epoch 19, Validation Loss: -32.62
Epoch 20, Validation Loss: -33.67
Epoch 21, Validation Loss: -33.64
Epoch 22, Validation Loss: -33.68
Epoch 23, Validation Loss: -33.38
Epoch 24, Validation Loss: -34.01
Epoch 25, Validation Loss: -33.54
Epoch 26, Validation Loss: -33.87
Epoch 27, Validation Loss: -33.60
Epoch 28, Validation Loss: -33.47
Epoch 29, Validation Los

In [21]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_pr_200-200.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:17<00:00,  5.86it/s]


            Obj Val  Constraints Viol  Elapsed Time
count  1.000000e+02        100.000000    100.000000
mean   4.175566e+01          0.531382      0.000939
std    6.427101e-14          0.338466      0.000517
min    4.175566e+01          0.000000      0.000000
25%    4.175566e+01          0.321623      0.000999
50%    4.175566e+01          0.488768      0.001000
75%    4.175566e+01          0.699078      0.001009
max    4.175566e+01          1.399221      0.002338
Number of infeasible solution: 95


### STE Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hsize = 16            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmQuadratic
from src.func.layer import netFC
from src.func import roundSTEModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
rnd = roundSTEModel(param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], int_ind=model.int_ind, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmQuadratic(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 5004.20
Epoch 1, Validation Loss: 240.90
Epoch 2, Validation Loss: 6.62
Epoch 3, Validation Loss: -0.59
Epoch 4, Validation Loss: -7.12
Epoch 5, Validation Loss: -12.77
Epoch 6, Validation Loss: -16.38
Epoch 7, Validation Loss: -19.77
Epoch 8, Validation Loss: -20.77
Epoch 9, Validation Loss: -21.48
Epoch 10, Validation Loss: -22.44
Epoch 11, Validation Loss: -23.62
Epoch 12, Validation Loss: -22.95
Epoch 13, Validation Loss: -24.06
Epoch 14, Validation Loss: -24.20
Epoch 15, Validation Loss: -24.51
Epoch 16, Validation Loss: -24.20
Epoch 17, Validation Loss: -25.13
Epoch 18, Validation Loss: -23.85
Epoch 19, Validation Loss: -24.36
Epoch 20, Validation Loss: -24.31
Epoch 21, Validation Loss: -24.59
Epoch 22, Validation Loss: -24.85
Epoch 23, Validation Loss: -24.17
Epoch 24, Validation Loss: -25.26
Epoch 25, Validation Loss: -25.18
Epoch 26, Validation Loss: -24.44
Epoch 27, Validation Loss: -24.62
Epoch 28, Validation Loss: -25.29
Epoch 29, Validation Loss: 

In [11]:
from src.heuristic import naive_round
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"][:100]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval_rel, _ = model.get_val()
    xval, objval = naive_round(xval_rel, model)
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_st_200-200.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.65it/s]


          Obj Val  Constraints Viol  Elapsed Time
count  100.000000             100.0    100.000000
mean    46.770966               0.0      0.002237
std      2.395742               0.0      0.001222
min     41.627415               0.0      0.000990
25%     45.375845               0.0      0.001587
50%     46.664422               0.0      0.002009
75%     47.832128               0.0      0.002511
max     55.029455               0.0      0.010712
Number of infeasible solution: 0
